## Проверка работы модели на тестовых данных

In [13]:
import pandas as pd
import numpy as np
from sklearn.metrics import roc_auc_score, classification_report
import dill
dill._dill._reverse_typemap['ClassType'] = type

### Загрузка и обзор тестового датасета

In [14]:
df = pd.read_csv('../datasets/test.csv', index_col=0)
df.head()

,id,Gender,Customer Type,Age,Type of Travel,Class,Flight Distance,Inflight wifi service,Departure/Arrival time convenient,Ease of Online booking,...,Inflight entertainment,On-board service,Leg room service,Baggage handling,Checkin service,Inflight service,Cleanliness,Departure Delay in Minutes,Arrival Delay in Minutes,satisfaction
0,19556,Female,Loyal Customer,52,Business travel,Eco,160,5,4,3,...,5,5,5,5,2,5,5,50,44.0,satisfied
1,90035,Female,Loyal Customer,36,Business travel,Business,2863,1,1,3,...,4,4,4,4,3,4,5,0,0.0,satisfied
2,12360,Male,disloyal Customer,20,Business travel,Eco,192,2,0,2,...,2,4,1,3,2,2,2,0,0.0,neutral or dissatisfied
3,77959,Male,Loyal Customer,44,Business travel,Business,3377,0,0,0,...,1,1,1,1,3,1,4,0,6.0,satisfied
4,36875,Female,Loyal Customer,49,Business travel,Eco,1182,2,3,4,...,2,2,2,2,4,2,4,0,20.0,satisfied


In [15]:
df.shape

(25976, 24)

### Проверка модели на тестовых данных
Разбиваю датасет на X и y

In [16]:
X = df.drop('satisfaction', axis=1)
y = pd.DataFrame({'satisfaction': df['satisfaction']})

Заменяю значения целевого признака на 1 и 0 для последущего расчета метрик качества модели

In [17]:
y.satisfaction = np.where(y.satisfaction == 'satisfied', 1, 0)

Загружаю модель из файла и делаю предсказания

In [18]:
with open('../models/xgbclf_pipeline.dill', 'rb') as in_strm:
    pipeline = dill.load(in_strm)

In [19]:
predictions = pipeline.predict(X)
pd.DataFrame({'preds': predictions}).to_csv("../datasets/test_predictions.csv", index=None)

In [23]:
binary_preds = np.where(predictions == 'satisfied', 1, 0)

In [24]:
print(f'ROC AUC score: {roc_auc_score(y, binary_preds)}')

ROC AUC score: 0.961491592533332


In [25]:
print(f'Classification report:\n{classification_report(y, binary_preds)}')

Classification report:
              precision    recall  f1-score   support

           0       0.96      0.98      0.97     14573
           1       0.97      0.94      0.96     11403

    accuracy                           0.96     25976
   macro avg       0.97      0.96      0.96     25976
weighted avg       0.96      0.96      0.96     25976

